In [12]:
import pandas as pd
import numpy as np
import faiss

In [13]:
# View Config
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 1000)
pd.set_option('max_colwidth', 400)

In [15]:
# Read embeddings
index = faiss.read_index('index')

In [16]:
df = pd.read_csv('goodreads_data.csv')

In [17]:
def create_textual_representation(row):
    return f"""Book: {row['Book']}
Author: {row['Author']}
Genres: {row['Genres']}
Description: {row['Description']}
"""


In [18]:
df['textual_representation'] = df.apply(create_textual_representation, axis=1)

In [19]:
df.sample()

,Unnamed: 0,Book,Author,Description,Genres,Avg_Rating,Num_Ratings,URL,textual_representation
3672,3672,"Daemon (Daemon, #1)",Daniel Suarez,"Technology controls almost everything in our modern-day world, from remote entry on our cars to access to our homes, from the flight controls of our airplanes to the movements of the entire world economy. Thousands of autonomous computer programs, or daemons, make our networked world possible, running constantly in the background of our lives, trafficking e-mail, transferring money, and monito...","['Science Fiction', 'Fiction', 'Thriller', 'Cyberpunk', 'Audiobook', 'Mystery', 'Technology']",4.16,"44,401",https://www.goodreads.com/book/show/4699575-daemon,"Book: Daemon (Daemon, #1)\nAuthor: Daniel Suarez\nGenres: ['Science Fiction', 'Fiction', 'Thriller', 'Cyberpunk', 'Audiobook', 'Mystery', 'Technology']\nDescription: Technology controls almost everything in our modern-day world, from remote entry on our cars to access to our homes, from the flight controls of our airplanes to the movements of the entire world economy. Thousands of autonomous c..."


In [20]:
df[df.Book.str.contains('To Kill')]

,Unnamed: 0,Book,Author,Description,Genres,Avg_Rating,Num_Ratings,URL,textual_representation
0,0,To Kill a Mockingbird,Harper Lee,"The unforgettable novel of a childhood in a sleepy Southern town and the crisis of conscience that rocked it. ""To Kill A Mockingbird"" became both an instant bestseller and a critical success when it was first published in 1960. It went on to win the Pulitzer Prize in 1961 and was later made into an Academy Award-winning film, also a classic.Compassionate, dramatic, and deeply moving, ""To Kill ...","['Classics', 'Fiction', 'Historical Fiction', 'School', 'Literature', 'Young Adult', 'Historical']",4.27,"5,691,311",https://www.goodreads.com/book/show/2657.To_Kill_a_Mockingbird,"Book: To Kill a Mockingbird\nAuthor: Harper Lee\nGenres: ['Classics', 'Fiction', 'Historical Fiction', 'School', 'Literature', 'Young Adult', 'Historical']\nDescription: The unforgettable novel of a childhood in a sleepy Southern town and the crisis of conscience that rocked it. ""To Kill A Mockingbird"" became both an instant bestseller and a critical success when it was first published in 1960..."
439,439,Harper Lee's To Kill a Mockingbird,Harold Bloom,"Published in 1960 and awarded the Pulitzer Prize in 1961, To Kill a Mockingbird is required reading for many middle and high school students. The coming-of-age tale of its young narrator, Jean Louise ""Scout"" Finch, of Maycomb, Alabama, is interwoven with explorations of the issues of prejudice, innocence, compassion, and hypocrisy. This accessible study guide is a compilation of important curr...","['Classics', 'Fiction', 'Historical Fiction', 'Literature', 'High School', 'School', 'Literary Criticism']",4.43,"7,508",https://www.goodreads.com/book/show/24374.Harper_Lee_s_To_Kill_a_Mockingbird,"Book: Harper Lee's To Kill a Mockingbird\nAuthor: Harold Bloom\nGenres: ['Classics', 'Fiction', 'Historical Fiction', 'Literature', 'High School', 'School', 'Literary Criticism']\nDescription: Published in 1960 and awarded the Pulitzer Prize in 1961, To Kill a Mockingbird is required reading for many middle and high school students. The coming-of-age tale of its young narrator, Jean Louise ""Sc..."
7685,7685,A Guide To To Kill A Mockingbird,Mary Hartley,"TheLiterature Made Easy Series is more than just plot summaries. Each book describes a classic novel and drama by explaining themes, elaborating on characters, and discussing each author's unique literary style, use of language, and point of view. Extensive illustrations and imaginative, enlightening use of graphics help to make each book in this series livelier, easier, and more fun to use th...",[],4.24,143,https://www.goodreads.com/book/show/301235.A_Guide_To_To_Kill_A_Mockingbird,"Book: A Guide To To Kill A Mockingbird\nAuthor: Mary Hartley\nGenres: []\nDescription: TheLiterature Made Easy Series is more than just plot summaries. Each book describes a classic novel and drama by explaining themes, elaborating on characters, and discussing each author's unique literary style, use of language, and point of view. Extensive illustrations and imaginative, enlightening use of ..."


In [21]:
fav = df.iloc[0]

In [23]:
print(fav['Book'])

To Kill a Mockingbird


In [24]:
from pymilvus import model

embedding_fn = model.DefaultEmbeddingFunction()

In [25]:
print(fav['textual_representation'])

Book: To Kill a Mockingbird
Author: Harper Lee
Genres: ['Classics', 'Fiction', 'Historical Fiction', 'School', 'Literature', 'Young Adult', 'Historical']
Description: The unforgettable novel of a childhood in a sleepy Southern town and the crisis of conscience that rocked it. "To Kill A Mockingbird" became both an instant bestseller and a critical success when it was first published in 1960. It went on to win the Pulitzer Prize in 1961 and was later made into an Academy Award-winning film, also a classic.Compassionate, dramatic, and deeply moving, "To Kill A Mockingbird" takes readers to the roots of human behavior - to innocence and experience, kindness and cruelty, love and hatred, humor and pathos. Now with over 18 million copies in print and translated into forty languages, this regional story by a young Alabama woman claims universal appeal. Harper Lee always considered her book to be a simple love story. Today it is regarded as a masterpiece of American literature.



In [26]:
query_vectors = embedding_fn.encode_queries([fav['textual_representation']])

In [27]:
embedding = np.array(query_vectors, dtype='float32')

In [28]:
# Return top 5 most similar results
D, I = index.search(embedding, 5)  # D - Distance, I - Index

In [29]:
best_matches = np.array(df['textual_representation'])[I.flatten()]

In [30]:
for i in best_matches:
    print('Next Book:')
    print(i)
    print()

Next Book:
Book: To Kill a Mockingbird
Author: Harper Lee
Genres: ['Classics', 'Fiction', 'Historical Fiction', 'School', 'Literature', 'Young Adult', 'Historical']
Description: The unforgettable novel of a childhood in a sleepy Southern town and the crisis of conscience that rocked it. "To Kill A Mockingbird" became both an instant bestseller and a critical success when it was first published in 1960. It went on to win the Pulitzer Prize in 1961 and was later made into an Academy Award-winning film, also a classic.Compassionate, dramatic, and deeply moving, "To Kill A Mockingbird" takes readers to the roots of human behavior - to innocence and experience, kindness and cruelty, love and hatred, humor and pathos. Now with over 18 million copies in print and translated into forty languages, this regional story by a young Alabama woman claims universal appeal. Harper Lee always considered her book to be a simple love story. Today it is regarded as a masterpiece of American literature.


N